In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch_geometric.utils as geo_utils

from utils import *

In [2]:
df_syn = load_synergyage()
df_biogrid = load_biogrid()

In [3]:
def encode_intervention_class(intervention_class: str) -> torch.Tensor:        
    if intervention_class == "NS":
        code = [0, 1, 0]
    elif intervention_class == "PRO":
        code = [0, 0, 1]
    elif intervention_class == "ANTI":
        code = [1, 0 ,0]
    else:
        raise ValueError(f"Intervention class {intervention_class} not recognized.")
    return torch.as_tensor(code).type(torch.LongTensor)


# 0) Clean df_synergy (current setup):
#    a) Remove genes not in biogrid
biogrid_genes = get_biogrid_gene_names(df_biogrid)
# synergyage_genes = get_synergyage_genes(df_syn)
selected = []
for i, row in df_syn.iterrows():
    genes = row["genes_str"].split(";")
    row_has_gene_not_in_biogrid = False
    for gene in genes:
        if gene not in biogrid_genes:
            row_has_gene_not_in_biogrid = True
            break
    if row_has_gene_not_in_biogrid:
        continue
    else:
        selected.append(i)
df_syn = df_syn.iloc[selected, :].copy()
#    b) remove duplicated interventions (later)

# 1) Shuffle df_synergy
df_syn = df_syn.iloc[np.random.permutation(len(df_syn)).tolist()].copy()


In [17]:
# 2) Create array of graphs and target tensors
data_list = []
for i, row in df_syn.iterrows():
    intervention_genes = row['genes_str'].split(";")
    class_encoded: torch.Tensor = encode_intervention_class( row["LIFESPAN_CLASS"] )

    # For removing the nodes that are KO
    # biogrid_mask = ( (~df_biogrid[GENE_NAME_A_COL].isin(intervention_genes)) & (~df_biogrid[GENE_NAME_B_COL].isin(intervention_genes)))
    # df_biogrid_trimmed = df_biogrid.loc[biogrid_mask].copy()
    
    G = nx.from_pandas_edgelist(df_biogrid[[GENE_NAME_A_COL, GENE_NAME_B_COL]])
    data = geo_utils.from_networkx(G)
    data.x = None
    raise NotImplementedError
    data.y = class_encoded
    data_list.append(data)

# loader = DataLoader(data_list, batch_size = 4)